In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet50

In [4]:
# Set the main folder path
main_folder = r"C:\Users\RIYA DHAWAS\OneDrive\Desktop\Internship\archive (1)"


In [5]:
# Load the ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [22]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras 
# Set the feature extraction function
from skimage.feature import graycomatrix, graycoprops
import csv
from torchvision.models import resnet50
import pandas as pd 
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn

In [14]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras 
# Set the main folder path
main_folder = r"C:\Users\RIYA DHAWAS\OneDrive\Desktop\Internship\archive (1)" 

def preprocess_image(image):
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    # Resize the image
    image = cv2.resize(image, (256, 256)) 
    return image
    

# Set the feature extraction function
from skimage.feature import graycomatrix, graycoprops
def extract_features(image): 
    l=[]
    # Load pre-trained Faster R-CNN model
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the image to a tensor
    image_tensor = torch.from_numpy(image_rgb.transpose((2, 0, 1))).float()

    # Run the image through the model
    output = model([image_tensor])

    # Retrieve the predicted bounding boxes
    boxes = output[0]['boxes'].detach().numpy()

    # Step 2: Shadow Removal and Enhancing Banana Regions

    # Convert the image to the LAB color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Split the LAB image into channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # Apply adaptive thresholding to the L channel to obtain a binary mask of shadows
    _, shadow_mask = cv2.threshold(l_channel, 50, 255, cv2.THRESH_BINARY)

    # Invert the shadow mask
    shadow_mask_inv = cv2.bitwise_not(shadow_mask)

    # Apply the shadow mask to the L channel to reduce shadow effects
    l_channel_no_shadow = cv2.bitwise_and(l_channel, shadow_mask_inv)

    # Combine the modified L channel with the original A and B channels
    image_lab_no_shadow = cv2.merge([l_channel_no_shadow, a_channel, b_channel])

    # Convert the modified LAB image back to the BGR color space
    image_no_shadow = cv2.cvtColor(image_lab_no_shadow, cv2.COLOR_LAB2BGR)

    # Step 3: Segment Banana Bunches using K-means Clustering

    # Convert the image without shadow to the LAB color space
    image_no_shadow_lab = cv2.cvtColor(image_no_shadow, cv2.COLOR_BGR2LAB)

    # Extract the L channel
    l_channel_no_shadow = cv2.split(image_no_shadow_lab)[0]

    # Reshape the L channel to a 2D array of pixels
    pixels = l_channel_no_shadow.reshape(-1, 1)

    # Convert the pixel values to floating point
    pixels = np.float32(pixels)

    # Perform K-means clustering
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    k = 2
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Reshape the labels to match the original image shape
    segmented = labels.reshape(image.shape[:2])

    # Create a mask for the banana region
    mask = np.uint8(segmented == 1) * 255

    # Apply the mask to the original image
    segmented_image = cv2.bitwise_and(image, image, mask=mask)

    # Step 4: Extract Shape-based Features

    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through each contour and extract shape-based features
    for contour in contours:
        # Compute contour properties
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        # Calculate bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)

        # Calculate aspect ratio
        aspect_ratio = float(w) / h

        # Calculate convex hull
        hull = cv2.convexHull(contour)

        # Calculate solidity if hull area is non-zero
        hull_area = cv2.contourArea(hull)
        if hull_area > 0:
            solidity = area / hull_area
        else:
            solidity = 0  
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_uint8 = gray.astype(np.uint8)  # Convert to unsigned integer type
    
    # Compute the GLCM
    distances = [1]  # Distance between pixels (you can experiment with different values)
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Angles for texture measurements
    glcm = graycomatrix(gray_uint8, distances, angles, levels=256, symmetric=True, normed=True)
    
    # Extract texture features
    contrast = graycoprops(glcm, prop='contrast').ravel()
    dissimilarity = graycoprops(glcm, prop='dissimilarity').ravel()
    homogeneity = graycoprops(glcm, prop='homogeneity').ravel()
    energy = graycoprops(glcm, prop='energy').ravel()
    correlation = graycoprops(glcm, prop='correlation').ravel() 
    color_contrast = np.max(image) - np.min(image)
    red_channel = image[:,:,0]
    green_channel = image[:,:,1]
    blue_channel = image[:,:,2]
    blue_channel[blue_channel == 255] = 0
    green_channel[green_channel == 255] = 0
    red_channel[red_channel == 255] = 0

    red_mean = np.mean(red_channel)
    green_mean = np.mean(green_channel)
    blue_mean = np.mean(blue_channel)

    red_std = np.std(red_channel)
    green_std = np.std(green_channel)
    blue_std = np.std(blue_channel)
    x = pd.Series()
    x['Area'] = area
    x['Perimeter'] = perimeter
    x['Aspect Ratio'] = aspect_ratio
    x['Solidity'] = solidity
    x['Contrast'] = contrast
    x['Dissimilarity'] = dissimilarity
    x['Homogeneity'] = homogeneity
    x['Energy'] = energy
    x['Correlation'] = correlation
    x['Red Mean'] = red_mean
    x['Blue Mean'] = blue_mean
    x['Green Mean'] = green_mean
    x['Red Std'] = red_std
    x['Green Std'] = green_std
    x['Blue Std'] = blue_std
    x['Color Contrast'] = color_contrast
    return x

In [6]:
# Define the output CSV file path
csv_file = 'feat_5_7.csv'

In [13]:
# Define the main folder containing the class folders
main_folder = r"C:\Users\RIYA DHAWAS\OneDrive\Desktop\Internship\archive (1)" 

# Define the CSV file path
csv_file = r'C:\Users\RIYA DHAWAS\OneDrive\Desktop\Internship\main_feat.csv'

In [51]:
i=0
# Create ResNet50 model
resnet_model = resnet50(pretrained=True)

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Get the number of ResNet50 features
    resnet_feature_count = resnet_model.fc.out_features

    # Create the header row
    header_row = ['Image', 'Class']  # Add 'Class' column to the header row
    header_row += [f'ResNet50_Feature{i+1}' for i in range(resnet_feature_count)]
    # Manually add the custom feature names to the header row
    header_row += ['Area', 'Perimeter','Aspect Ratio','Solidity','Contrast','Dissimilarity','Homogeneity','Energy','Correlation','Red Mean','Blue Mean','Green Mean','Red Std','Green Std','Blue Std','Color Contrast']  # Add your custom feature names here

    # Write the header row
    writer.writerow(header_row)
    
    # Iterate through each folder (class) in the main folder
    for class_folder in os.listdir(main_folder):
        class_folder_path = os.path.join(main_folder, class_folder)

        # Iterate through each image file in the class folder
        for image_file in os.listdir(class_folder_path):
            image_path = os.path.join(class_folder_path, image_file)

            # Load and preprocess the image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Resize the image to the desired input size of the ResNet model
            image = cv2.resize(image, (224, 224))

            # Convert the image to a PyTorch tensor
            image_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()
            
            # Normalize the image tensor
            image_tensor = image_tensor / 255.0
            normalize = torch.tensor([0.485, 0.456, 0.406]).reshape(1, -1, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225]).reshape(1, -1, 1, 1)
            image_tensor = (image_tensor - normalize) / std

            # Extract features using ResNet50
            with torch.no_grad():
                resnet_features = resnet_model(image_tensor)
                resnet_features = resnet_features.squeeze().numpy()

            # Extract custom features using your code
            custom_features = extract_features(image)  # Use the modified feature extraction code

            # Convert the features to a list
            class_name = class_folder  # Get the class name from the subfolder
            features_list = [image_file, class_name] + resnet_features.tolist() + [custom_features['Area'], custom_features['Perimeter'],custom_features['Aspect Ratio'],custom_features['Solidity'],custom_features['Contrast'],custom_features['Dissimilarity'],custom_features['Homogeneity'],custom_features['Energy'],custom_features['Correlation'],custom_features['Red Mean'],custom_features['Blue Mean'],custom_features['Green Mean'],custom_features['Red Std'],custom_features['Green Std'],custom_features['Blue Std'],custom_features['Color Contrast']]  # Add your custom feature values here

            # Write the features to the CSV file
            writer.writerow(features_list)
            i+=1
            print(i)

print("CSV file created successfully.")


C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1


C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2


C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

1930


C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1931


C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1932
CSV file created successfully.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import KNNImputer

In [3]:
# from sklearn_lightgbm import LGBMClassifier


In [4]:
# Load the CSV data into a DataFrame
data = pd.read_csv( r'C:\Users\RIYA DHAWAS\OneDrive\Desktop\Internship\main_feat.csv')

# Drop the 'image_file' column
data.drop('Image', axis=1, inplace=True)

# Split the data into features (X) and target (y)
X = data.drop('Class', axis=1)
y = data['Class']

# Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [5]:
for i in X.columns:
    X[i]=pd.to_numeric(X[i],errors='coerce')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Create an instance of the KNN imputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer to fill missing values
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [8]:
pipelines = {
    'Decision Tree': Pipeline([('clf', DecisionTreeClassifier())]),
    'Random Forest': Pipeline([('clf', RandomForestClassifier())]),
    'SVM': Pipeline([('clf', SVC())]),
    'XGBoost': Pipeline([('clf', XGBClassifier())]),
    'Multi-Layer Perceptron': Pipeline([('clf', MLPClassifier())])
}


In [9]:
results = {}

for name, pipeline in pipelines.items():
    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)

    # Store the results
    results[name] = {
        'Accuracy': accuracy,
        'Classification Report': report
    }


C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\RIYA DHAWAS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\RIYA DHAWAS\anaconda3\lib\si

In [10]:
for name, result in results.items():
    print(f"--- {name} ---")
    print(f"Accuracy: {result['Accuracy']:.4f}")
    print("Classification Report:")
    print(pd.DataFrame(result['Classification Report']).transpose())
    print()

--- Decision Tree ---
Accuracy: 0.6770
Classification Report:
               precision    recall  f1-score     support
Apple(1-5)      0.529412  0.692308  0.600000   26.000000
Apple(10-14)    0.800000  0.835821  0.817518   67.000000
Apple(5-10)     0.800000  0.666667  0.727273   36.000000
Banana(1-5)     0.416667  0.416667  0.416667   12.000000
Banana(10-15)   0.631579  0.705882  0.666667   17.000000
Banana(15-20)   0.864865  0.941176  0.901408   34.000000
Banana(5-10)    0.500000  0.500000  0.500000   16.000000
Carrot(1-2)     0.529412  0.642857  0.580645   14.000000
Carrot(3-4)     0.000000  0.000000  0.000000    5.000000
Expired         0.819149  0.726415  0.770000  106.000000
Tomato(1-5)     0.409091  0.562500  0.473684   16.000000
Tomato(10-15)   0.238095  0.384615  0.294118   13.000000
Tomato(5-10)    0.538462  0.333333  0.411765   21.000000
carrot(5-6)     0.000000  0.000000  0.000000    4.000000
accuracy        0.677003  0.677003  0.677003    0.677003
macro avg       0.505481  

In [12]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an instance of the KNN imputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer to fill missing values
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test) 

# Scale the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the target variable to categorical (one-hot) encoding
num_classes = len(label_encoder.classes_)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Create a neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

Epoch 1/10
49/49 [==============================] - 1s 8ms/step - loss: 1.3788 - accuracy: 0.5877 - val_loss: 0.6674 - val_accuracy: 0.8165
Epoch 2/10
49/49 [==============================] - 0s 3ms/step - loss: 0.4239 - accuracy: 0.8880 - val_loss: 0.3868 - val_accuracy: 0.8837
Epoch 3/10
49/49 [==============================] - 0s 5ms/step - loss: 0.2211 - accuracy: 0.9443 - val_loss: 0.3483 - val_accuracy: 0.8966
Epoch 4/10
49/49 [==============================] - 0s 4ms/step - loss: 0.1475 - accuracy: 0.9625 - val_loss: 0.2694 - val_accuracy: 0.9096
Epoch 5/10
49/49 [==============================] - 0s 4ms/step - loss: 0.1206 - accuracy: 0.9638 - val_loss: 0.2614 - val_accuracy: 0.9096
Epoch 6/10
49/49 [==============================] - 0s 4ms/step - loss: 0.0933 - accuracy: 0.9638 - val_loss: 0.2740 - val_accuracy: 0.9096
Epoch 7/10
49/49 [==============================] - 0s 4ms/step - loss: 0.0927 - accuracy: 0.9696 - val_loss: 0.2777 - val_accuracy: 0.9147
Epoch 8/10
49/49 [==